Basic demonstration of the quantum chemistry code I have written. It seems to be fairly quick, but uses quite a bit of ram. One trick which I certainly did not exploit is that some parts of the environments are simply the harmitian conjugate of other parts, and the environments can therefore be compressed further (and should be twice as cheap to calculate).

In [1]:
using Revise,LinearAlgebra, MPSKit, TensorKit, MPSKitExperimental

LinearAlgebra.BLAS.set_num_threads(1);
#TensorKit.TO.disable_cache();
TensorKit.Strided.disable_threads();


(ex.head, ex.args) = (:||, Any[:(numout(v) == 2 && numin(v) == 1), :(throw(TensorOperations.IndexError("Incorrect number of input-output indices for v: (2, 1) instead of ($((numout(v), numin(v)))).")))])
(ex.head, ex.args) = (:||, Any[:(numout(A) == 2 && numin(A) == 1), :(throw(TensorOperations.IndexError("Incorrect number of input-output indices for A: (2, 1) instead of ($((numout(A), numin(A)))).")))])
(ex.head, ex.args) = (:||, Any[:(numout(O) == 2 && numin(O) == 2), :(throw(TensorOperations.IndexError("Incorrect number of input-output indices for O: (2, 2) instead of ($((numout(O), numin(O)))).")))])
(ex.head, ex.args) = (:||, Any[:(numout(Ab) == 2 && numin(Ab) == 1), :(throw(TensorOperations.IndexError("Incorrect number of input-output indices for Ab: (2, 1) instead of ($((numout(Ab), numin(Ab)))).")))])
(ex.head, ex.args) = (:call, Any[:(TensorOperations.promote_contract), :(TensorOperations.scalartype(v)), :(TensorOperations.scalartype(A))])
(ex.head, ex.args) = (:call, Any[:(Te

FCIDUMP is a well known (though badly documented) format to store the electronic integrals. I have written an experimental parser to load in such files:

In [2]:
(ERI,K,E0,NORB,NELEC,MS2) = parse_fcidump("N2.CCPVDZ.FCIDUMP")

([1.1486574428316065 + 0.0im 0.09862429942213358 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.09862429942213358 + 0.0im 0.01772802215378584 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im … 0.009036810489967113 + 0.0im 0.003653054088686124 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … 0.003653054088686124 + 0.0im 0.0017533811663050644 + 0.0im;;; 0.09862429942213358 + 0.0im 0.413306086967678 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.01772802215378584 + 0.0im -0.002611637416218654 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im … -0.00533684879644213 + 0.0im 0.002227779206976413 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … -0.001273700295269154 + 0.0im -0.0005596210416448279 + 0.0im;;; 0.03622757258934596 + 0.0im -0.0470908045818145 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.0013749347300627281 + 0.0im 0.004411244674669814 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im … -2.8000517192292658e-5 + 0.0im -0.006213347982843338 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … -0.000871798277289064 + 

ERI is the Electronic Repulsion Integral, K is the two-body term, E0 is an energy shift, NORB is the number of orbitals (mps length), NELEC is the number of electrons and MS2 is the total spin.

fcidump files can contain orbital symmetries, but I ignore these.

In [3]:
(ham,_) = fused_quantum_chemistry_hamiltonian(E0,K,ERI,Float64);

The second return value contains some metadata which can be ignored. Instantiate the state:

In [4]:
psp = Vect[(Irrep[U₁]⊠Irrep[SU₂] ⊠ FermionParity)]((0,0,0)=>1, (1,1//2,1)=>1, (2,0,0)=>1);
left = Vect[(Irrep[U₁]⊠Irrep[SU₂] ⊠ FermionParity)]((-NELEC,MS2//2,mod(-NELEC,2))=>1);
right = oneunit(left);

virtual = Vect[(Irrep[U₁]⊠Irrep[SU₂] ⊠ FermionParity)]((i,s,b)=>1 for i in -NELEC:0, s in 0:1//2:(MS2//2+1), b in (0,1));

st = FiniteMPS(rand,Float64,NORB,psp,virtual;left,right);

We also need to instantiate the environments, because MPSKit will otherwise call the default environment constructor which keeps everything in ram (I think I even disabled that constructor). To create an environment that lives on disk, we can use:

In [5]:
envs = disk_environments(st,ham);

Let's call DMRG!

In [8]:
@time (st,envs) = find_groundstate(st,ham, DMRG2(trscheme=truncdim(100), verbose = true, maxiter=3),envs);

In [6]:
@time (st,envs) = find_groundstate(st,ham, DMRG2(trscheme=truncdim(100), verbose = true, maxiter=3),envs);

┌ Info: DMRG2 iteration:
│   iter = 1
│   ϵ = 0.9999999994239843
│   λ = -109.10798776824517
│   Δt = 144.880029555
└ @ MPSKit /home/maarten/projects/MPSKit.jl/src/algorithms/groundstate/dmrg.jl:119
┌ Info: DMRG2 iteration:
│   iter = 2
│   ϵ = 0.0029276584349371415
│   λ = -109.23611889333257
│   Δt = 36.778272979
└ @ MPSKit /home/maarten/projects/MPSKit.jl/src/algorithms/groundstate/dmrg.jl:119
┌ Info: DMRG2 iteration:
│   iter = 3
│   ϵ = 0.0004006089262981183
│   λ = -109.24516655944437
│   Δt = 32.987991949
└ @ MPSKit /home/maarten/projects/MPSKit.jl/src/algorithms/groundstate/dmrg.jl:119
┌ Warning: DMRG2 maximum iterations
│   iter = 3
│   ϵ = 0.0004006089262981183
│   λ = -109.24516655944437
└ @ MPSKit /home/maarten/projects/MPSKit.jl/src/algorithms/groundstate/dmrg.jl:123
┌ Info: DMRG2 summary:
│   ϵ = 0.0004006089262981183
│   λ = -109.24516655944437
│   Δt = 219.983143374
└ @ MPSKit /home/maarten/projects/MPSKit.jl/src/algorithms/groundstate/dmrg.jl:127


231.667762 seconds (4.47 G allocations: 208.782 GiB, 18.50% gc time, 96.89% compilation time)


In [10]:
#@time (st,envs) = find_groundstate(st,ham,DMRG(maxiter=3),envs);

In [8]:
Revise.retry()

(ex.head, ex.args) = (:tuple, Any[Symbol("##2793"), Symbol("##2794")])
(ex.head, ex.args) = (:tuple, Any[:(numout(v)), :(numin(v))])
(ex.head, ex.args) = (:||, Any[:(numout(v) == 2 && (TensorKit.numin)(v) == 1), :(throw(TensorOperations.IndexError("incorrect number of input-output indices: (2, 1) instead of " * string((var"##2793", var"##2794")) * " for v.")))])
(ex.head, ex.args) = (:tuple, Any[Symbol("##2795"), Symbol("##2796")])
(ex.head, ex.args) = (:tuple, Any[:(numout(A)), :(numin(A))])
(ex.head, ex.args) = (:||, Any[:(numout(A) == 2 && (TensorKit.numin)(A) == 1), :(throw(TensorOperations.IndexError("incorrect number of input-output indices: (2, 1) instead of " * string((var"##2795", var"##2796")) * " for A.")))])
(ex.head, ex.args) = (:tuple, Any[Symbol("##2797"), Symbol("##2798")])
(ex.head, ex.args) = (:tuple, Any[:(numout(O)), :(numin(O))])
(ex.head, ex.args) = (:||, Any[:(numout(O) == 2 && (TensorKit.numin)(O) == 2), :(throw(TensorOperations.IndexError("incorrect number of i

┌ Error: Failed to revise /home/maarten/projects/MPSKitExperimental.jl/src/quantumchemistry/fused_env.jl
│   exception = Revise.ReviseEvalException("/home/maarten/projects/MPSKitExperimental.jl/src/tightloop/tightloop.jl:214", ErrorException("invalid redefinition of constant left_trans"), Any[(top-level scope at tightloop.jl:214, 1)])
└ @ Revise /home/maarten/.julia/packages/Revise/sCmwS/src/packagedef.jl:722
